In [1]:
import os

In [2]:
%pwd

'c:\\Users\\91931\\Text-Summarization-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\91931\\Text-Summarization-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from textSummarization.constants import *
from textSummarization.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [9]:
from transformers import TrainingArguments, Trainer

# helps in preparing the inputs, attention masks, and labels in the correct format.
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, T5Tokenizer, T5ForConditionalGeneration
from datasets import load_dataset, load_from_disk
import torch

In [10]:
torch.cuda.empty_cache()

In [11]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = T5Tokenizer.from_pretrained(self.config.model_ckpt)
        model_t5 = T5ForConditionalGeneration.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_t5)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

# Params.yaml file read
        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=2 , fp16=True,  gradient_checkpointing=True
        ) 
        # Gradient accumulation reduced 16 -> 8 -> 2

        
        


        trainer = Trainer(model=model_t5, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_t5.save_pretrained(os.path.join(self.config.root_dir,"t5-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

        # os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


In [13]:
import torch
print(torch.cuda.is_available())  # Should return True if CUDA is available
print(torch.cuda.device_count())  # Should return the number of GPUs available
print(torch.cuda.get_device_name(0))  # Should print the name of the GPU


True
1
NVIDIA GeForce GTX 1650


In [14]:
# Do Legacy = False -- Havent done yet  
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-08-26 09:40:58,416: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-26 09:40:58,419: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-26 09:40:58,420: INFO: common: created directory at: artifacts]
[2024-08-26 09:40:58,422: INFO: common: created directory at: artifacts/model_trainer]


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
c:\Users\91931\Text-Summarization-Project\virtual\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\91931\Text-Summarization-Project\virtual\Lib\site-packages\accelerate\accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
  0%|

{'loss': 16.4981, 'grad_norm': 85.5018539428711, 'learning_rate': 6.000000000000001e-07, 'epoch': 0.02}


  5%|▍         | 20/409 [00:42<13:25,  2.07s/it]

{'loss': 16.6054, 'grad_norm': 108.46190643310547, 'learning_rate': 1.4000000000000001e-06, 'epoch': 0.05}


...

{'loss': 0.7194, 'grad_norm': 5.110471725463867, 'learning_rate': 3.82e-05, 'epoch': 0.95}


 98%|█████████▊| 400/409 [13:48<00:18,  2.07s/it]

{'loss': 0.5756, 'grad_norm': 1.3574801683425903, 'learning_rate': 3.9200000000000004e-05, 'epoch': 0.98}


100%|██████████| 409/409 [14:08<00:00,  2.08s/it]


{'train_runtime': 848.907, 'train_samples_per_second': 0.965, 'train_steps_per_second': 0.482, 'train_loss': 5.97938637628532, 'epoch': 1.0}
